<a href="https://colab.research.google.com/github/faranhaider1/MNIST/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision.datasets import MNIST
import torchvision.transforms.functional as TF
import numpy as np
from torchvision import transforms
from PIL import Image
from torch import nn, optim, save, load

In [5]:
# Custom function to resize and crop images to a target size of 20x20 pixels
def resize_and_crop_image(image, target_size=(20, 20)):
    # Resizes the given image to the specified dimensions
    resized_image = TF.resize(image, target_size)
    return resized_image

# Custom data loader function to manually process, batch, and load data
def custom_data_loader(dataset, batch_size, shuffle=True):
    # Creates an array of indices for the dataset, allowing for optional shuffling
    indices = np.arange(len(dataset))
    if shuffle:
        # Shuffles indices to randomize data order
        np.random.shuffle(indices)

    # Iterates over the dataset in batches of specified size
    for start_idx in range(0, len(dataset), batch_size):
        batch_indices = indices[start_idx:start_idx + batch_size]
        batch_images = []
        batch_labels = []

        # Processes each image in the current batch
        for i in batch_indices:
            image, label = dataset[i]
            # Applies resizing and converts the image to a tensor
            image = resize_and_crop_image(image)
            batch_images.append(TF.to_tensor(image))
            batch_labels.append(label)

        # Converts lists of images and labels to tensor batches
        batch_images_tensor = torch.stack(batch_images)
        batch_labels_tensor = torch.tensor(batch_labels)

        # Yields a batch of images and labels as tensors
        yield batch_images_tensor, batch_labels_tensor

# Image Classifier Model with Custom Architecture
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        # Convolutional layers with padding to maintain image size
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # Third layer with adjusted input channels to accommodate feature stacking
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64 + 32, 64, kernel_size=3, padding=1)
        # Fully connected layer, assuming input size is 20x20 pixels post-resizing
        self.fc = nn.Linear(64 * 20 * 20, 10)

    def forward(self, x):
        #Add the input of the first layer to its output
        x1 = F.relu(self.conv1(x))

        # Resize original input to match x1 dimensions if necessary and add
        x1_added = x1 + F.interpolate(x, size=x1.size()[2:])

        # Pass result through second convolutional layer
        x2 = F.relu(self.conv2(x1_added))

        # Task 5: Stack the feature map from x2 with x1_added for the third layer
        x2_stacked = torch.cat((x2, x1_added), dim=1)

        # Pass stacked feature maps through third convolutional layer
        x3 = F.relu(self.conv3(x2_stacked))

        # Flatten the output for the fully connected layer
        x_flattened = torch.flatten(x3, 1)

        # Calculate class scores
        out = self.fc(x_flattened)
        return out

# Initialize model, loss function, and optimizer
model = ImageClassifier().to('cpu')
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Training loop
if __name__ == "__main__":
    # Load MNIST dataset without any initial transformations
    train_dataset = MNIST(root="data", train=True, download=True, transform=None)

    # Specify batch size and initialize custom data loader
    batch_size = 32
    epochs = 10
    custom_loader = custom_data_loader(train_dataset, batch_size)

    # Train for a specified number of epochs
    for epoch in range(epochs):
        for batch_images, batch_labels in custom_loader:
            # Zero gradients, perform forward pass, compute loss, and backpropagate
            optimizer.zero_grad()
            outputs = model(batch_images)
            loss = loss_fn(outputs, batch_labels)
            loss.backward()
            optimizer.step()

        # Print loss after each epoch for monitoring
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.006955201271921396
Epoch 2/10, Loss: 0.006955201271921396
Epoch 3/10, Loss: 0.006955201271921396
Epoch 4/10, Loss: 0.006955201271921396
Epoch 5/10, Loss: 0.006955201271921396
Epoch 6/10, Loss: 0.006955201271921396
Epoch 7/10, Loss: 0.006955201271921396
Epoch 8/10, Loss: 0.006955201271921396
Epoch 9/10, Loss: 0.006955201271921396
Epoch 10/10, Loss: 0.006955201271921396


In [45]:
# For simplicity, this snippet assumes the model is already trained
# and focuses on the saving, loading, and inference parts

if __name__ == "__main__":
    # Save the model state
    model_path = '/content/data/model_state.pt'  # Adjusted path for compatibility with this environment
    with open(model_path, 'wb') as f:
        save(model.state_dict(), f)

    # Load the model state
    with open(model_path, 'rb') as f:
        model.load_state_dict(load(f))

    # Prepare an image for inference
    img_path = '/content/img_2.jpg'  # uploading image
    img = Image.open(img_path)

    # Assuming the image is grayscale, similar to MNIST dataset images
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((20, 20)),  # Resize to match the input size of the model
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # Assuming normalization similar to the training phase
    ])

    img_tensor = transform(img).unsqueeze(0).to('cpu')  # Add batch dimension and move to the correct device

    # Perform inference
    output = model(img_tensor)
    predicted_class = torch.argmax(output, dim=1)
    print(f"Predicted class: {predicted_class.item()}")

Predicted class: 0
